In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import os
import sys
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..\..'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)

In [37]:
from funcs import *

## Read Data + Save

In [4]:
human_data(file='data/BIOGRID-ORGANISM-Homo_sapiens-4.4.204.tab3.txt',
           save_file='data/homo_preprocess.tsv')

In [5]:
disease_data(file='data/curated_gene_disease_associations.tsv',
             disease_id='C0003873')

## Data Overview

In [6]:
data_overview(human_file='data/homo_preprocess.tsv',
              disease_file='data/diseaseC0003873.tsv')

Number of genes associated with the desease: 174
Classes of the desease: ['C17;C05;C20']
Number of genes present in the interactome: 173
Largest connected component: 83


## Algorithms

#### MCL

In [14]:
MCL_hyper(human_file='data/homo_preprocess.tsv', start=18, end=28)

inflation: 2.7 modularity: 0.469805545634236


In [15]:
cluster = MCL(human_file='data/homo_preprocess.tsv', inflation=1.8)

#### Diffusion Heat

In [38]:
diff_dict = diffusion_heat(path='diffusion_heat/', top=50)

In [68]:
for key, val in diff_dict.items():
    
    with open('data/seed'+key+'val.txt', 'r') as f:
        ground_truth = [line.strip() for line in f]
        
    multibinarizer = MultiLabelBinarizer()
    
    val = [str(i) for i in val]

    val_new = multibinarizer.fit(val).transform(val)
    ground_truth_new = multibinarizer.transform(ground_truth)
        
    metric = metrics(test_set=val_new, ground_truth=ground_truth_new)

ValueError: Found input variables with inconsistent numbers of samples: [17, 50]

In [57]:
from sklearn.metrics import recall_score, precision_score, f1_score, ndcg_score

In [58]:
ndcg_score(ground_truth, val)

ValueError: Found input variables with inconsistent numbers of samples: [17, 50]

In [60]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import recall_score, precision_score

A=[['a','b','c'],['d','e','f','g']]
B=[['a','b'],['d','f','g']]

multibinarizer = MultiLabelBinarizer()

A_new = multibinarizer.fit(A).transform(A)
B_new = multibinarizer.transform(B)

#average = 'valid strategy for multilabel
precision_score(A_new,B_new,average='samples')
#output -> 1.0
recall_score(A_new, B_new, average='samples')
#output -> 0.70833333333333326

0.7083333333333333

In [61]:
A_new

array([[1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 1]])

In [62]:
B_new

array([[1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1]])

In [15]:
DIAMOND(network_file='data/PPI.txt', seed_file='data/seed_genes.txt', n=200, alpha=1, out_file='data/diamond_results.txt')

DIAMOnD(): ignoring 1 of 174 seed genes that are not in the network

 results have been saved to 'data/diamond_results.txt' 



In [61]:
DIABLE(network_file='data/PPI.txt', seed_file='data/seed_genes.txt', n=200, alpha=1, out_file='data/diable_results.txt')

DiaBLE(): ignoring 1 of 174 seed genes that are not in the network

 results have been saved to 'data/diable_results.txt' 



In [91]:
RANDOM_WALK_WITH_RESTART(network_file='data/PPI.txt', seed_file='data/seed_genes.txt', r=0.1, score_thr=0, tol=1e-6)

RANDOM_WALK_WITH_RESTART(): ignoring 1 of 174 seed genes that are not in the network

 results have been saved to 'data/random_walk_wr_results.txt' 



## 10-fold cross validation

In [33]:
from imported_code.diamond import DIAMOnD

In [43]:
gt = np.array([34, 14, 12, 41, 23, 2])
pred = np.array(np.arange(1, 100))
n = 27

kwargs = {
    'network_file':'data/PPI.txt', 
    'seed_file':'data/seed_genes.txt',
    'func_args':{
        'max_number_of_added_nodes':180,
        'alpha':1,
        'outfile':'./data/test_kfold_diamond.txt'
    }
    
}

metrics = k_fold(DIAMOnD, compute_metrics, 3, **kwargs)

/home/stefano/Documents/University/Master/Second_year/BI/homeworks/BI-Project/funcs.py:491: RuntimeWarning: invalid value encountered in double_scalars
  return metrics


In [44]:
metrics

defaultdict(list,
            {'precision_at_50': (0.02, 0.016329931618554522),
             'recall_at_50': (0.017241379310344827, 0.014077527257374586),
             'ndcg_at_k_50': (0.020747046136719956, 0.015646240006390748),
             'F1-score_at_50': (0.01851851851851852, 0.015120307054217151),
             'precision_at_17': (0.0392156862745098, 0.0277296776935901),
             'recall_at_17': (0.011494252873563218, 0.008127664151569512),
             'ndcg_at_k_17': (0.03175479080952405, 0.022518169531146874),
             'F1-score_at_17': (0.017777777777777778, 0.01257078722109418),
             'precision_at_43': (0.023255813953488372, 0.018988292579714558),
             'recall_at_43': (0.017241379310344827, 0.014077527257374586),
             'ndcg_at_k_43': (0.022981040787214534, 0.01733099147627673),
             'F1-score_at_43': (0.019801980198019802, 0.016168249127281705),
             'precision_at_86': (0.023089332526719093, 0.00805731519419522),
             '

## Extended validation